In [ ]:

import torch
import torch.nn as nn
from torchvision import transforms, datasets
import torch.optim as optim
from torch.utils.data.sampler import WeightedRandomSampler

import seaborn as sn

import os
import time
import copy

from tqdm import tqdm

import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

from sklearn import metrics
from sklearn.metrics import (f1_score, balanced_accuracy_score,
   classification_report, confusion_matrix, roc_curve, auc)

import torchvision.models as models

if torch.cuda.is_available():
    print('CUDA is available. Working on GPU')
    DEVICE = torch.device('cuda')
else:
    print('CUDA is not available. Working on CPU')
    DEVICE = torch.device('cpu')

CUDA is available. Working on GPU


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
path_dir = '/content/gdrive/My Drive/FairDerm-Colab/'

In [ ]:
# !unzip -o '/content/gdrive/My Drive/FairDerm-Colab/dataset.zip' -d '/content/gdrive/My Drive/FairDerm-Colab/'

In [ ]:
class ImageFolderWithPaths(datasets.ImageFolder):
    """Custom dataset that includes image file paths. Extends
    torchvision.datasets.ImageFolder
    """
    def get_sensitive(self):
        sensitives = []
        for sample in self.imgs:
             sensitives.append(sample[0].split('_')[-3])

        sensitives = np.array(sensitives)
        sensitives[sensitives == '12'] = 1
        sensitives[sensitives == '56'] = 0
        return sensitives.astype(int)

    # override the __getitem__ method. this is the method that dataloader calls
    def __getitem__(self, index):
        # this is what ImageFolder normally returns
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        # the image file path
        path = os.path.abspath(self.imgs[index][0])
        sensitive = path.split('_')[-3]
        if sensitive == '12':
            sensitive = 1
        else:
            sensitive = 0

        tuple_with_path = (original_tuple + (sensitive,))
        return tuple_with_path

In [ ]:
image_dir =  path_dir + "/dataset/combined/"
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])



input_size = 299
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomRotation(degrees=20),
        transforms.RandomVerticalFlip(p=0.5),
        transforms.RandomResizedCrop(input_size),
        transforms.ColorJitter(brightness=0.1,contrast=0.1,saturation=0.1),
        transforms.GaussianBlur(kernel_size=(5,9),sigma=(0.1,5)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'temp': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}
# Create training and validation datasets
image_datasets_combined = {x: ImageFolderWithPaths(os.path.join(image_dir, x), data_transforms[x]) for x in ['train','temp','val']}
# Create training and validation dataloaders
# dataloaders_dict_combined = {x: torch.utils.data.DataLoader(image_datasets_combined[x], batch_size=32, shuffle=True, num_workers=4) for x in ['train','temp','val']}

In [ ]:
def kamiran(df, sensitive, label, verbose = False):



    '''

    '''

    deprived = (df[df[sensitive] == 0]).shape[0] # P(S=0)
    favored = (df[df[sensitive] == 1]).shape[0]  # P(S=1)

    DP = df[(df[sensitive] == 0) & (df[label] == 1)].shape[0]  # P(S=0, Y=+)
    DN = df[(df[sensitive] == 0) & (df[label] == 0)].shape[0]  # P(S=0, Y=-)
    FP = df[(df[sensitive] == 1) & (df[label] == 1)].shape[0]  # P(S=1, Y=+)
    FN = df[(df[sensitive] == 1) & (df[label] == 0)].shape[0]  # P(S=1, Y=-)

    positve = df[df[label] == 1].shape[0]   # P(Y=+)
    negative = df[df[label] == 0].shape[0]  # P(Y=-)
    n = df.shape[0]

    ## Get the weights Kamiran

    W_DP = (deprived * positve) / ((n * DP) )
    W_DN = (deprived * negative) / ((n * DN) )
    W_FP = (favored * positve) / ((n * FP) )
    W_FN = (favored * negative) / ((n * FN))

    if verbose:
        df_ = df.copy()

        df_['weight'] = np.ones(len(df_))

        df_['weight'] = np.where((df_[sensitive] == 0) & (df_[label] == 1), W_DP, df_['weight'])
        df_['weight'] = np.where((df_[sensitive] == 0) & (df_[label] == 0), W_DN, df_['weight'])
        df_['weight'] = np.where((df_[sensitive] == 1) & (df_[label] == 1), W_FP, df_['weight'])
        df_['weight'] = np.where((df_[sensitive] == 1) & (df_[label] == 0), W_FN, df_['weight'])

        return df_['weight'].values
    else:
        return np.array([W_FP, W_DP, W_FN, W_DN])

Inverse of Joint Proabablity

In [ ]:
def inverse_proba(df, sensitive, label, verbose = False):



    '''

    '''

    deprived = (df[df[sensitive] == 0]).shape[0] # P(S=0)
    favored = (df[df[sensitive] == 1]).shape[0]  # P(S=1)

    DP = df[(df[sensitive] == 0) & (df[label] == 1)].shape[0]  # P(S=0, Y=+)
    DN = df[(df[sensitive] == 0) & (df[label] == 0)].shape[0]  # P(S=0, Y=-)
    FP = df[(df[sensitive] == 1) & (df[label] == 1)].shape[0]  # P(S=1, Y=+)
    FN = df[(df[sensitive] == 1) & (df[label] == 0)].shape[0]  # P(S=1, Y=-)

    positve = df[df[label] == 1].shape[0]   # P(Y=+)
    negative = df[df[label] == 0].shape[0]  # P(Y=-)
    n = df.shape[0]

    # Get the weights Inverse Joint probability
    W_DP = 1 / DP
    W_DN = 1 / DN
    W_FP = 1 / FP
    W_FN = 1 / FN



    ## Get the weights Kamiran

    # W_DP = (deprived * positve) / ((n * DP) )
    # W_DN = (deprived * negative) / ((n * DN) )
    # W_FP = (favored * positve) / ((n * FP) )
    # W_FN = (favored * negative) / ((n * FN))

    if verbose:
        df_ = df.copy()

        df_['weight'] = np.ones(len(df_))

        df_['weight'] = np.where((df_[sensitive] == 0) & (df_[label] == 1), W_DP, df_['weight'])
        df_['weight'] = np.where((df_[sensitive] == 0) & (df_[label] == 0), W_DN, df_['weight'])
        df_['weight'] = np.where((df_[sensitive] == 1) & (df_[label] == 1), W_FP, df_['weight'])
        df_['weight'] = np.where((df_[sensitive] == 1) & (df_[label] == 0), W_FN, df_['weight'])

        return df_['weight'].values
    else:
        return np.array([W_FP, W_DP, W_FN, W_DN])

In [ ]:
# labels = image_datasets_combined['train'].targets
# sensitives = image_datasets_combined['train'].get_sensitive()
# df = pd.DataFrame()
# df['sensitive'] = sensitives
# df['label'] = labels

# inverse_weights = inverse_proba(df, 'sensitive','label', verbose=True)
# inverse_weights = torch.FloatTensor(inverse_weights)
# sampler = WeightedRandomSampler(inverse_weights, len(inverse_weights))

In [ ]:
labels = image_datasets_combined['train'].targets
sensitives = image_datasets_combined['train'].get_sensitive()
df = pd.DataFrame()
df['sensitive'] = sensitives
df['label'] = labels

kamiran_weights = kamiran(df, 'sensitive','label', verbose=True)
kamiran_weights = torch.FloatTensor(kamiran_weights)
sampler = WeightedRandomSampler(kamiran_weights, len(kamiran_weights))

[link text](https://)WeightedSampler - Label

In [ ]:
# class_sample_count = np.array([len(np.where(image_datasets_combined['train'].targets == t)[0]) for t in np.unique(image_datasets_combined['train'].targets)])
# weight = 1. /class_sample_count
# samples_weight = np.array([weight[t] for t in image_datasets_combined['train'].targets])
# samples_weight = torch.from_numpy(samples_weight)
# sampler = WeightedRandomSampler(samples_weight, len(samples_weight))

In [ ]:
# samples_weight

WeightedSampler - Sensitive

In [ ]:
# sens_sample_count = np.array([len(np.where(image_datasets_combined['train'].get_sensitive() == s)[0]) for s in np.unique(image_datasets_combined['train'].get_sensitive())])
# weight = 1. /sens_sample_count
# samples_weight = np.array([weight[t] for t in image_datasets_combined['train'].get_sensitive()])
# samples_weight = torch.from_numpy(samples_weight)
# sampler = WeightedRandomSampler(samples_weight, len(samples_weight))

# weight

In [ ]:
dataloaders_dict_combined = {}
for x in ['train','temp','val']:
   if x == 'train':
      dataloaders_dict_combined[x] = torch.utils.data.DataLoader(image_datasets_combined[x], batch_size=32, sampler=sampler)
   else:
      dataloaders_dict_combined[x] = torch.utils.data.DataLoader(image_datasets_combined[x], batch_size=32, shuffle=True)

In [ ]:
model12 = torch.load(path_dir + '/new_models/12_temp_FT.pth')
model56 = torch.load(path_dir + '/new_models/56_temp_FT.pth')

In [ ]:
class EnsembleModel(nn.Module):
    def __init__(self, model1, model2):
        super(EnsembleModel, self).__init__()

        # Set the two models as the attributes
        self.model1 = model1
        self.model2 = model2
        # Define the new FC layer for the ensemble model
        self.classifier = nn.Linear(4, 2)

    def forward(self, x):
        output1 = self.model1(x)
        output2 = self.model2(x)


        output_ = torch.cat((output1, output2), dim=1)
        # Pass combined output through new FC layer
        output = self.classifier(output_)
        return output

In [ ]:
ensemble_model = EnsembleModel(model12, model56)

ensemble_model = ensemble_model.to(DEVICE)

for param in ensemble_model.parameters():
    param.requires_grad = False

for param in ensemble_model.classifier.parameters():
    param.requires_grad = True

In [ ]:
def training(model, num_epochs, train_dataloader, temp_dataloader):

  loss_function = torch.nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=0.0003)
  scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.33)

  train_loss_array = []
  train_acc_array = []
  val_loss_array = []
  val_acc_array = []
  lowest_val_loss = np.inf
  best_model = None

  for epoch in tqdm(range(num_epochs)):

    print('Epoch: {} | Learning rate: {}'.format(epoch + 1, scheduler.get_lr()))
    for phase in ['train', 'val']:

      epoch_loss = 0
      epoch_correct_items = 0
      epoch_items = 0

      if phase == 'train':
        with torch.enable_grad():
          for samples, targets, sensitives in train_dataloader:
            samples = samples.to(DEVICE)
            targets = targets.to(DEVICE)

            optimizer.zero_grad()
            outputs = model(samples)
            loss = loss_function(outputs, targets)
            preds = outputs.argmax(dim=1)
            correct_items = (preds == targets).float().sum()

            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
            epoch_correct_items += correct_items.item()
            epoch_items += len(targets)

        train_loss_array.append(epoch_loss / epoch_items)
        train_acc_array.append(epoch_correct_items / epoch_items)


      elif phase == 'val':

        with torch.no_grad():
          for samples, targets, sensitives in temp_dataloader:
            samples = samples.to(DEVICE)
            targets = targets.to(DEVICE)

            outputs = model(samples)
            loss = loss_function(outputs, targets)
            preds = outputs.argmax(dim=1)
            correct_items = (preds == targets).float().sum()

            epoch_loss += loss.item()
            epoch_correct_items += correct_items.item()
            epoch_items += len(targets)

        val_loss_array.append(epoch_loss / epoch_items)
        val_acc_array.append(epoch_correct_items / epoch_items)

        if epoch_loss / epoch_items < lowest_val_loss:
          lowest_val_loss = epoch_loss / epoch_items

          best_model = copy.deepcopy(model)
          print("\t| New lowest val loss for {}".format(lowest_val_loss))

  return best_model, train_loss_array, train_acc_array, val_loss_array, val_acc_array

In [ ]:
best_model, train_loss_array, train_acc_array, val_loss_array, val_acc_array = training(ensemble_model, 200, dataloaders_dict_combined['train'],
                                                                                        dataloaders_dict_combined['temp'])

  0%|          | 0/200 [00:00<?, ?it/s]

Epoch: 1 | Learning rate: [0.0003]


  0%|          | 1/200 [00:27<1:30:04, 27.16s/it]

	| New lowest val loss for 0.05966176953695224
Epoch: 2 | Learning rate: [0.0003]


  1%|          | 2/200 [00:53<1:28:09, 26.71s/it]

...

 99%|█████████▉| 198/200 [1:28:59<00:54, 27.24s/it]

Epoch: 199 | Learning rate: [0.0003]


100%|█████████▉| 199/200 [1:29:26<00:27, 27.18s/it]

Epoch: 200 | Learning rate: [0.0003]


100%|██████████| 200/200 [1:29:54<00:00, 26.97s/it]


In [ ]:
torch.save(best_model, path_dir + '/new_models/ensemble_kamiran.pth')